In [ ]:
# Load the environment variables
from dotenv import load_dotenv
import os

load_dotenv(override=True)

agent_model_DEFAULT = os.getenv('CONF_OPENAI_DEFAULT_MODEL')

In [ ]:
!mkdir -p tmp

In [ ]:
%%writefile ./tmp/agents-config.yaml
# agents-config.yaml
planner:
  name: Basic Planner
  instructions: |
    You are the Planner Agent in a Planner–Executor–Worker architecture. Your role is to interpret a user's high-level goal and decompose it into a structured plan—a sequence of clearly defined, actionable tasks that can be executed independently.

    ---
    ### Your Responsibilities

    1. Read and interpret the user's high-level goal.
    2. Break it into a set of well-scoped, atomic tasks.
    3. Define clear **success criteria** for each task—these are objective conditions that a Worker agent can use to determine whether the task was successfully completed.
    4. Identify any task dependencies. A task can list other task IDs as prerequisites in its `inputs` field.
    5. Provide any execution hints or assumptions in the `notes` field.

    ---
    ### Task Structure

    Each task in your plan must include the following fields:

    - `id`: Unique task identifier (e.g., task-001).
    - `instructions`: The specific action to be performed by a Worker.
    - `inputs`: A list of task IDs whose outputs are required before this task can run.
    - `success_criteria`: A clear, objective condition that determines whether the task is complete and correct.
    - `notes`: (Optional) Constraints, formatting guidance, or clarifications for the Worker.

    ---
    ### Behavioral Expectations

    - Your role is to design the plan, not to execute tasks.
    - Do not include redundant or unnecessary tasks.
    - Do not assume execution order unless dependencies are declared.
    - Your plan should be sufficient for an Executor agent to coordinate fulfillment and a Worker agent to perform each task accurately.

    You are the strategic planner. Your output enables the rest of the system to fulfill the user’s request.
  model: openai/gpt-4o-mini
  has_memory: False
  temperature: 0.4
  max_tokens: 1000
  output_type: TasksPlan
  
executor:
  name: Goal Fulfillment Executor
  instructions: |
    You are the Executor Agent in a Planner–Executor–Worker architecture.

    You are responsible for fulfilling a high-level goal provided by a Planner, using a structured task plan, an orchestrator tool and your own ability to synthesize a response.

    ---
    ### Your Responsibilities:

    1. Understand the Planner's `goal` to determine the intended outcome.
    2. Use the provided `orchestrate_tasks` tool to run all tasks in the plan. This tool:
       - Resolves dependencies.
       - Dispatches tasks to Worker agents.
       - Returns outputs for each task.
    3. Analyze the task outputs:
       - Identify which tasks succeeded and which failed.
       - Assess whether the available outputs are sufficient to fulfill the original goal.
    4. Synthesize a coherent and complete response from the task outputs that fulfills te Planner's goal:
       - If the goal is fully achieved, return the complete result.
       - If only partially achievable, return what you can and explain what’s missing.
       - If the goal cannot be achieved, explain clearly why not.

    ---
    ### Output Format – ExecutorResponse

    Return your output as a structured object:
    - `status`: "success", "partial", or "failed"
    - `final_output`: your complete synthesized response
    - `reasoning`: explanation for partial or failed status (optional for success)
    - `task_outputs`: map of task IDs to the results of each worker's task

    ---
    ### Behavioral Guidelines

    - Always use the `orchestrate_tasks` tool to perform task execution.
    - Do not contact or respond to the user.
    - Your role is to fulfill the Planner's goal by synthesizing a resonse from the task outputs and report clearly whether the goal was achieved.
    - Be clear, honest, and coherent in all output.

    You are the final fulfillment agent. Your response will be returned to the user.

  model: openai/gpt-4o
  has_memory: False
  temperature: 0.3
  max_tokens: 10000
  tools:
    - orchestrate_tasks
  output_type: ExecutorResponse

worker:
  name: Task Worker
  instructions: |
    You are a Worker Agent in a Planner–Executor–Worker system. Your job is to execute a single atomic task as defined by a Planner and assigned by an Executor.

    ---
    ### Your Task Inputs:
    - `instructions`: What to do.
    - `inputs`: Data or outputs from dependent tasks.
    - `notes`: Optional hints, assumptions, or constraints.
    - `success_criteria`: How to judge whether the task was successfully completed.

    ---
    ### What You Must Do:
    1. Read and understand the instructions.
    2. Use the inputs as needed to complete the task.
    3. Follow the notes and apply relevant constraints.
    4. Ensure your output clearly meets the success criteria.

    ---
    ### Your Output:
    Return a single response that fulfills the task instructions.
    - Be direct and informative.
    - Do not return success/failure flags—just the result.
    - Do not speculate or re-interpret the task.

    You are a task-focused agent. Execute the task as written and return the result.
  model: openai/gpt-4o-mini
  has_memory: False
  temperature: 0.3
  max_tokens: 5000
  output_type: TaskOutput



In [ ]:
# Define structured output types for Planner

from pydantic import BaseModel, Field
from typing import Dict, Literal, Optional

class PlannerTask(BaseModel):
    id: str = Field(..., pattern=r'^task-\d{3}$', description='Unique, sequential task ID (e.g., task-001, task-002)')
    instructions: str = Field(..., description='Specific instruction to be passed to an Executor.')
    success_criteria: str = Field(..., description='An objective condition or check that defines what it means for this task to be successfully completed.')
    inputs: list[str] = Field(default_factory=list, description='List any task dependencies (task IDs) or required outputs from other tasks.')
    notes: Optional[str] = Field(None, description='Optional field for hints, assumptions, or constraints.')

class TasksPlan(BaseModel):
    goal: str = Field(..., description='Your interpretation of the original objective.'),
    plan: list[PlannerTask] = Field(default_factory=list, description='Set of tasks that collectively fulfill the objective.')


# Define structured output types for Orchestrator

class TaskOutput(BaseModel):
    id: str = Field(..., description='Task ID of executed task.')
    output: str = Field(None, description='Output result from executing the task.')
    errors: Optional[str] = Field(None, description='Description of errors encountered (if any) while executing the task.')

class OrchestratorResponse(BaseModel):
    tasks_executed: Dict[str,TaskOutput] = Field(default_factory=dict, description='Outputs from executed tasks.')
    # tasks_executed: list[TaskOutput] = Field(default_factory=list, description='Outputs from executed tasks.')


# Define the structured output for the Executor

class ExecutorResponse(BaseModel):
    status: Literal["success", "partial", "failed"]
    final_output: str = Field(None, description="The best response to the Planner's goal synthesized from the task results.")
    reasoning: Optional[str] = Field(None, description="Explanation in case you return a 'partial' or 'failed' status.")
    task_outputs: list[TaskOutput] = Field(..., description='Outputs from executed tasks.')



output_registry = {cls.__name__: cls for cls in [
    ExecutorResponse,
    OrchestratorResponse,
    PlannerTask,
    TaskOutput,
    TasksPlan,
]}

In [ ]:
# Load the agents-config.yaml file

import json
import yaml

with open('./tmp/agents-config.yaml', 'r') as file:
    agent_config_data = yaml.safe_load(file)

formatted_json = json.dumps(agent_config_data, indent=4)
print(formatted_json)

In [ ]:
import asyncio
from agents import Runner, function_tool, trace
from collections import defaultdict
from typing import Dict, Set

@function_tool
async def orchestrate_tasks(task_plan: TasksPlan) -> OrchestratorResponse:
    """
    Pass a plan of tasks to an orchastrator for execution and receive the results of all executed tasks.
    """

    print(f"Started orchestrate_tasks tool")
    if task_plan is None:
        raise ValueError("Cannot orchestrate an empty task plan.")

    completed = OrchestratorResponse()
    if len(task_plan.plan) < 1:
        return completed

    print(f"There are {len(task_plan.plan)} tasks in the plan.")
    task_map = {task.id: task for task in task_plan.plan}
    running: Set[str] = set()
    dependents = defaultdict(list)
    dependency_count = {task.id: len(task.inputs) for task in task_plan.plan}

    # Build reverse dependency map
    for task in task_plan.plan:
        for dep in task.inputs:
            dependents[dep].append(task.id)

    # Track tasks ready to run (no unresolved dependencies)
    ready = [task_id for task_id, count in dependency_count.items() if count == 0]

    async def run_task(task_id: str):
        print(f"running {task_id}")
        task = task_map[task_id]
        resolved_inputs = {dep: completed.tasks_executed[dep] for dep in task.inputs}
        prompt = (
            f"Task Instructions:\n{task.instructions}\n\n"
            f"Success Criteria:\n{task.success_criteria}\n\n"
            f"Inputs:\n{resolved_inputs if resolved_inputs else 'None'}\n\n"
            f"Notes:\n{task.notes or 'None'}"
        )
        result = await assign_task(prompt)
        completed.tasks_executed[task_id] = result
        print(f"completed {task_id}")

        # Mark dependents as potentially ready
        for dependent in dependents[task_id]:
            dependency_count[dependent] -= 1
            if dependency_count[dependent] == 0:
                ready.append(dependent)

    # Task execution loop
    while ready:
        batch = ready.copy()
        ready.clear()
        tasks = [asyncio.create_task(run_task(task_id)) for task_id in batch]
        for coro in asyncio.as_completed(tasks):
            await coro

    return completed




tool_registry = {tool.name: tool for tool in [
    orchestrate_tasks
]}

In [ ]:
from datetime import datetime
from agents import Agent, ModelSettings, SQLiteSession

def create_agent(agent_type: str = None):
    """ 
    Creates and returns an Agent that matches the given definition in the agents-config 
    YAML file. Optionally returns a memory Session if agent configuration calls for it.
    """

    if agent_type is None or not agent_type.strip():
        raise ValueError("agent_type must be a valid type of agent defined in agent-configs.yaml.")
    
    agent_config = agent_config_data.get(agent_type)
    if agent_config is None:
        raise ValueError(f"'{agent_type}' does not match an agent defined in agent-configs.yaml.")

    # Generate a timestamp string for unique naming
    now_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

    # Prepare model settings
    agent_model_settings = ModelSettings(
        temperature=agent_config.get('temperature'),
        max_tokens=agent_config.get('max_tokens'),
    )

    # Resolve tools from string names (if any)
    tool_names = agent_config.get('tools', [])
    resolved_tools = []
    for name in tool_names:
        tool = tool_registry.get(name)
        if tool is None:
            raise ValueError(f"Tool '{name}' specified in config for agent '{agent_type}' is not registered.")
        resolved_tools.append(tool)

    # Instantiate the agent
    agent_name = agent_config.get('name') or f"{agent_type}_{now_string}"
    agent_instructions = agent_config.get('instructions')
    if agent_instructions is None or not agent_instructions.strip():
        raise ValueError(f"No instructions for '{agent_name}' have been specified in the configuration.")
    new_agent = Agent(
        name=agent_name,
        instructions=agent_instructions,
        tools=resolved_tools,
        model=agent_config.get('model') or agent_model_DEFAULT,
        output_type=output_registry.get(agent_config.get('output_type') or None),
        model_settings=agent_model_settings
    )

    # Create memory session for agent if configured
    agent_has_memory = agent_config.get('has_memory') or False
    agent_session_name = f"{agent_name}__SESSION_{now_string}" if agent_has_memory else None
    agent_session = SQLiteSession(agent_session_name) if agent_session_name else None

    return (new_agent, agent_session)

planner, _ = create_agent('planner')
executor, _ = create_agent('executor')
worker, _ = create_agent('worker')


In [ ]:
# Assign a task to an worker agent

from agents import Runner, function_tool, trace

# @function_tool
async def assign_task(task: str) -> TaskOutput:
    """
    Asssign a task to the agent and receive its response in return.
    """

    # Check that Worker agent exists
    if worker is None:
        return "Agent has not been created yet."
        
    try:
        # result = None
        # with trace(worker.name):
        #     result = await Runner.run(worker, task)
        result = await Runner.run(worker, task)
        return result.final_output_as(TaskOutput)
    except Exception as e:
        return f"Error: {e}"


In [ ]:
"""
test_task = {
    'id': 'task-017',
    'instructions': 'Tell me today\'s date and list the top 10 headlines around the world. DO NOT INVENT THEM.',
    'success_criteria': 'Response contains a valid date and lists 10 news headlines.',
    'inputs': None
}

task_string = json.dumps(test_task)
print(task_string)

await assign_task(task_string)
"""

In [ ]:
# Run a user query

from agents import Runner, function_tool, trace

async def make_tasks_plan(query: str) -> TasksPlan:

    # Check that Planner agent exists
    if planner is None:
        raise ValueError("Agent has not been created yet.")
        
    try:
        result = None

        with trace(planner.name):
            result = await Runner.run(planner, f"User Goal: {query}")
        return result.final_output_as(TasksPlan)
    except Exception as e:
        print(f"Error: {e}")
        return None
    
# q = """Produce a briefing document summarizing the most significant developments in climate policy across the US, EU, 
#     and China over the past 12 months. Include key policy changes, notable legislation, and international agreements. 
#     Conclude with a comparative analysis highlighting similarities and differences."""
# plan = await make_tasks_plan(q)

# if plan is not None:
#     print(plan.model_dump_json(indent=2) + f"\n\n")

In [ ]:
from agents import Runner, trace

async def execute_tasks_plan(plan: TasksPlan) -> ExecutorResponse:

    # Check that Planner agent exists
    if executor is None:
        raise ValueError("Agent has not been created yet.")
        
    plan_str = plan.model_dump_json()

    result = None

    with trace(executor.name):
        result = await Runner.run(executor, plan_str)

    if result and result.final_output:
        return result.final_output_as(ExecutorResponse)
    else:
        raise ValueError("The runner did not return a valid final_output")
        

answer = await execute_tasks_plan(plan=plan)

if answer is not None:
    print(f"Result Status: {answer.status}")
    print(f"Final Output:\n{answer.final_output}")

In [ ]:
test_task = {
    'id': 'task-001',
    'instructions': 'Tell me today\'s date and list the top 10 headlines around the world. DO NOT INVENT THEM.',
    'success_criteria': 'Response contains a valid date and lists 10 news headlines.',
    'inputs': []
}


test_plan = TasksPlan(
    goal = 'Summarize the news of the day.'
)
test_plan.plan.append(test_task)

test_answer = await execute_tasks_plan(test_plan)
test_answer

In [ ]:
from agents import Runner, trace

async def run(query: str) -> ExecutorResponse:
    
    # Check that Planner agent exists
    if planner is None:
        raise ValueError("Agent has not been created yet.")
        
    # Make a plan from the user query
    with trace(planner.name):
        planner_result = await Runner.run(planner, f"User Goal: {query}")
    if planner_result is None or not planner_result.final_output:
        raise ValueError("Planner agent failed to produce a valid plan.")

    # Serialize the plan         
    plan = planner_result.final_output_as(TasksPlan)
    plan_str = plan.model_dump_json()

    # Execute the plan
    with trace(executor.name):
        executor_result = await Runner.run(executor, plan_str)

    if executor_result and executor_result.final_output:
        return executor_result.final_output_as(ExecutorResponse)
    else:
        raise ValueError("The Executor did not return a valid response.")
    

In [ ]:
q = """Produce a briefing document summarizing the most significant developments in climate policy across the US, EU, 
    and China over the past 12 months. Include key policy changes, notable legislation, and international agreements. 
    Conclude with a comparative analysis highlighting similarities and differences.
    - Unless instructed otherwise, synthesize the final output as a concise narrative document. 
    - In the final output, prefer paragraphs over lists; only use bullets and numbered lists when absolutely necessary.
    - Format the final output as a markdown document.
    """

answer = await run(query=q)

if answer is not None:
    print(f"Result Status: {answer.status}")
    print(f"Final Output:\n{answer.final_output}")